<h1><b>Tracing drain to ocean (ArcGIS6-0)</b></h1>

<h1>**Table of Contents**<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hydrologic-data-analysiss:-tracing-drain-to-ocean" data-toc-modified-id="Hydrologic-data-analysiss:-tracing-drain-to-ocean-1">Hydrologic data analysiss: tracing drain to ocean</a></span><ul class="toc-item"><li><span><a href="#No-Dumping---Drains-to-Ocean" data-toc-modified-id="No-Dumping---Drains-to-Ocean-1.1">No Dumping - Drains to Ocean</a></span></li><li><span><a href="#Getting-set-up" data-toc-modified-id="Getting-set-up-1.2">Getting set up</a></span></li><li><span><a href="#Create-watersheds-for-the-storm-drain" data-toc-modified-id="Create-watersheds-for-the-storm-drain-1.3">Create watersheds for the storm drain</a></span><ul class="toc-item"><li><span><a href="#Create-a-default-watershed-for-the-storm-drain-point-feature" data-toc-modified-id="Create-a-default-watershed-for-the-storm-drain-point-feature-1.3.1">Create a default watershed for the storm drain point feature</a></span></li><li><span><a href="#Create-a-watershed-on-the-nearest-drainage-line-using-a-search-distance" data-toc-modified-id="Create-a-watershed-on-the-nearest-drainage-line-using-a-search-distance-1.3.2">Create a watershed on the nearest drainage line using a search distance</a></span></li><li><span><a href="#Find-the-distance-from-the-storm-drain-point-to-the-snapped-point-on-the-nearest-drainage-line" data-toc-modified-id="Find-the-distance-from-the-storm-drain-point-to-the-snapped-point-on-the-nearest-drainage-line-1.3.3">Find the distance from the storm drain point to the snapped point on the nearest drainage line</a></span></li></ul></li><li><span><a href="#Find-the-downstream-flow-path-from-the-storm-drain" data-toc-modified-id="Find-the-downstream-flow-path-from-the-storm-drain-1.4">Find the downstream flow path from the storm drain</a></span><ul class="toc-item"><li><span><a href="#Inspect-the-length-of-downstream-trace-from-the-storm-drain-to-the-ocean-(which-ocean?)" data-toc-modified-id="Inspect-the-length-of-downstream-trace-from-the-storm-drain-to-the-ocean-(which-ocean?)-1.4.1">Inspect the length of downstream trace from the storm drain to the ocean (which ocean?)</a></span></li><li><span><a href="#Divide-the-downstream-trace-into-multiple-segments" data-toc-modified-id="Divide-the-downstream-trace-into-multiple-segments-1.4.2">Divide the downstream trace into multiple segments</a></span></li><li><span><a href="#Inspect-the-'from-distance'-and-'to-distance'-of-each-segment-in-the-downstream-trace." data-toc-modified-id="Inspect-the-'from-distance'-and-'to-distance'-of-each-segment-in-the-downstream-trace.-1.4.3">Inspect the 'from distance' and 'to distance' of each segment in the downstream trace.</a></span></li></ul></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-1.5">Conclusion</a></span></li></ul></li></ul></div>

<img src="img/creek_into_two_oceans.PNG">

# Hydrologic data analysis: tracing drain to ocean

This demo is based on a sample notebook from ESRI. The image above is from https://www.cntraveler.com/story/this-wyoming-creek-flows-into-two-oceans

A nice recent UI for a similar system is at https://river-runner-global.samlearner.com/ - try it!


## No Dumping - Drains to Ocean

Have you ever seen a sign near a storm drain that says "No Dumping - Drains to Ocean"? If you live inland, you might have seen one that says Drains to River, Lake, Bay, or Waterway. All of these are messages about the same thing—anything washed or dumped into the drain will be transported downstream and has the potential to pollute local waterways and the larger water bodies they connect to. For example, pollution carried down the Mississippi from Midwestern farms has created a dead zone in the Gulf of Mexico the size of Massachusetts where fish and plants have to fight for survival.

In this notebook, you'll learn how to find the area that drains to a storm drain, and the route that pollutants will take if they are dumped or washed into the drain. You'll find the upstream drainage area, called a watershed, for a storm drain near [a location of your choice] . Then you'll find the downstream flow path to where it empties into the Gulf of Mexico or another body of water. Knowing how to find these, you can experiment to find the watersheds and flow paths from other storm drain locations.

This notebook uses Spatial Analysis online service tools, **Create Watershed** and **Trace Downstream**, that perform analysis on your web GIS. You can find more information about the Create Watershed analysis tool [here](https://developers.arcgis.com/rest/analysis/api-reference/create-watersheds.htm) and the Trace Downstream analysis tool [here](https://developers.arcgis.com/rest/analysis/api-reference/trace-downstream.htm).

## Getting set up
Import the necessary libraries, and connect to your portal.

In [ ]:
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import arcgis
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureSet, FeatureCollection
from arcgis.geometry import Point, distance
from arcgis.features.analysis import create_watersheds
from arcgis.features.analysis import trace_downstream

# login with UCSD Single-Sign-On. 
gis=GIS("https://ucsdonline.maps.arcgis.com/home", client_id="bZshlNXFuaR2KHff") 


Create a new map and navigate to the area of interest: some area at the continental divide

In [ ]:
# storm_drain_map = gis.map("Teton Wilderness, WY, USA. ",8)
storm_drain_map = gis.map("Nebraska, USA. ",6)
storm_drain_map

In [ ]:
storm_drain_map.basemap.basemap = 'hybrid'

Let us pick a storm drain point for our analysis.

In [ ]:
storm_drain_feature = Feature.from_dict(
    {'geometry' : {'x' : -100.8046341,
                   'y' : 40.9163524,
                   'type' : 'point',
                   'spatialReference' : {'latestWkid' : 4326,
                   'wkid' : 4326}}})

# storm_drain_feature = Feature.from_dict(
#     {'geometry' : {'x' : -110.8046341,
#                    'y' : 43.9163524,
#                    'type' : 'point',
#                    'spatialReference' : {'latestWkid' : 4326,
#                    'wkid' : 4326}}})

storm_drain_df = FeatureSet([storm_drain_feature]).sdf
storm_drain_fc = storm_drain_df.spatial.to_feature_collection(
    'storm_drain_fc')

Plot the point on the map. In the following section you will build watersheds for this point.

In [ ]:
storm_drain_df.spatial.plot(map_widget = storm_drain_map, 
                            renderer_type ='s',
                            symbol_type = 'simple',
                            symbol_style = 'd', # d - for diamonds
                            colors = 'Reds_r',
                            cstep = 10,
                            outline_color = 'Blues',
                            marker_size = 20)

## Create watersheds for the storm drain

### Create a default watershed for the storm drain point feature
Use the Create Watershed tool from the `arcgis.feature.analysis` module to find the upstream contributing area to the storm drain point. You'll first create a watershed by keeping all the default parameters in the tool.

In [ ]:
try:
    storm_drain_watershed = create_watersheds(storm_drain_fc, gis=gis)
except Exception as e:
    print("Error: Portal not configured with Hydrology Utility")
    raise e

storm_drain_watershed

Add the storm drain watershed to the storm drain map.

In [ ]:
from arcgis.geometry import Point, Polygon
from arcgis.map.symbols import SimpleMarkerSymbolEsriSMS, SimpleFillSymbolEsriSFS

# Create symbol for pour points
pour_point_symbol = SimpleMarkerSymbolEsriSMS(
    style="esriSMSCircle",
    color=[0, 0, 255, 255],  # Blue
    size=8,
    outline={
        "color": [0, 0, 0, 255],
        "width": 1
    }
)

# Create symbol for watershed
watershed_symbol = SimpleFillSymbolEsriSFS(
    style="esriSFSSolid",
    color=[0, 255, 255, 128],  # Cyan with transparency
    outline={
        "color": [0, 0, 0, 255],
        "width": 1
    }
)

# Draw pour points
for feature in storm_drain_watershed['snap_pour_pts_layer'].query().features:
    point = Point(feature.geometry)
    storm_drain_map.content.draw(point, symbol=pour_point_symbol)

# Draw watershed polygons
for feature in storm_drain_watershed['watershed_layer'].query().features:
    polygon = Polygon(feature.geometry)
    storm_drain_map.content.draw(polygon, symbol=watershed_symbol)


Create a Spatially Enabled DataFrame from the watershed feature collection layer you created and read the records in this watershed.

In [ ]:
storm_drain_watershed_fc = storm_drain_watershed['watershed_layer']
storm_drain_watershed_fc.query().sdf

As you can see from the watershed feature layer collection added to the map and the `DataFrame` queried above, the size of this watershed is quite small. These results are not unusual. If your analysis point is located away from a drainage line, the resulting watershed is likely to be very small and not of much use in determining the upstream source of contamination. In most cases, your analysis point should be positioned on the nearest drainage line to find the watershed that flows to a point on the drainage line.

### Create a watershed on the nearest drainage line using a search distance

To find the closest drainage line, you can specify a **search distance**, which is what you’ll do in the next section. The results of this tool should be used to provide a regional understanding of the watershed rather than identify the exact location of the watershed at local scales. You'll do that next.

In [ ]:
storm_drain_large_watershed = create_watersheds(
    storm_drain_fc, 
    search_distance = 0.5, 
    search_units = "miles",
    context = {'outSR' : {'latestWkid' : 3857,
                          'wkid' : 102100}},
    gis=gis)

storm_drain_large_watershed

Query the large watershed created to determine the new analysis area.

In [ ]:
storm_drain_large_watershed_fc = \
    storm_drain_large_watershed['watershed_layer']
storm_drain_large_watershed_fc.query().sdf

In [ ]:
storm_drain_map.content.add(
    storm_drain_large_watershed['watershed_layer'],
    options = {'opacity' : 0.5})
storm_drain_map.content.add(
    storm_drain_large_watershed['snap_pour_pts_layer'],
    options = {'opacity' : 0.5})

Zoom out of storm drain map to see the watershed from the closest drainage line. When the analysis is completed, you’ll see that another watershed has been added to the map that encompasses nearly the entire smaller watershed. Zoom out of the map to see the entire watershed. 

## Find the downstream flow path from the storm drain

You've identified where the water in the storm drain comes from. In the next section, you'll find the path it will take to the ocean. You'll use the **Trace Downstream** tool to find the downstream flow path.

In [ ]:
storm_drain_downstream_trace = trace_downstream(
    input_layer=storm_drain_fc, gis=gis)

In [ ]:
downstream_map = gis.map('USA')
downstream_map

In [ ]:
downstream_map.basemap.basemap = 'gray-vector'


In [ ]:
downstream_map.content.add(
    storm_drain_large_watershed['watershed_layer'],
    options = {'opacity' : 0.5})
downstream_map.content.add(
    storm_drain_large_watershed['snap_pour_pts_layer'],
    options = {'opacity' : 0.5})



In [ ]:
from arcgis.map.symbols import SimpleLineSymbolEsriSLS

# Create downstream trace symbol (blue line)
trace_symbol = SimpleLineSymbolEsriSLS(
    style="esriSLSSolid",
    color=[0, 0, 255, 255],  # Blue
    width=2
)

# Draw downstream trace using query() for FeatureCollection
for feature in storm_drain_downstream_trace.query().features:
    downstream_map.content.draw(feature.geometry, symbol=trace_symbol)


### Inspect the length of downstream trace from the storm drain to the ocean (which ocean?)

In [ ]:
storm_drain_trace_sdf = storm_drain_downstream_trace.query().sdf
storm_drain_trace_sdf.head()

The trace is more than `1,200` miles long. 

### Divide the downstream trace into multiple segments

The trace line can be split into multiple lines where each line is of the specified length. For example, if there is a large contamination spill near the storm drain and the river it flows at a rate of `5` miles per hour, or `120` miles per day, you might want to split the river into `120` mile segments. Splitting the trace into `120` mile intervals will show approximately where the spill will travel each day. The line will be symbolized using graduated colors based on distance and will be labeled with the distance from the start of the trace. The resulting trace will have multiple line segments, each with fields `FromDistance` and `ToDistance`.

In [ ]:
storm_drain_downstream_trace_split = trace_downstream(
    input_layer = storm_drain_fc, gis=gis,
    split_distance = 120,
    split_units = 'Miles')

In [ ]:
downstream_map.content.add(storm_drain_downstream_trace_split)

### Inspect the 'from distance' and 'to distance' of each segment in the downstream trace.

In [ ]:
storm_drain_trace_split_sdf = \
    storm_drain_downstream_trace_split.query().sdf
storm_drain_trace_split_sdf.head(11)

## Conclusion
In this sample we used `create_watershed()` and `trace_downstream()` hydrology tools to derive watershed boundaries and the downstream flow path for a given point. The analysis can be easily repated for any point on the globe.

Our waterbodies are precious natural resources. Just as we saw from this sample, anything washed or dumped into the drain will be transported downstream and has the potential to pollute local waterways and the larger water bodies they connect to. 